<a href="https://colab.research.google.com/github/Ahsan-folium/ai-intern-week04-deep-learnin/blob/main/deepLearning_miniProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install transformers datasets
!pip install evaluate

In [2]:
from datasets import load_dataset
from transformers import (
AutoTokenizer,
AutoModelForSequenceClassification, TrainingArguments, Trainer
)

import numpy as np
import evaluate

In [3]:
# loading the imdb dataset
dataset = load_dataset("imdb")
print(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [4]:
# loading the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [5]:
# now we need to tokenize so each review text converts into tokens with ids
def tokenizer_function(dataset):
  return tokenizer(dataset["text"], padding=True, truncation=True)


tokenized_datasets = dataset.map(tokenizer_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [6]:
# now we load the model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# evaluation metrics
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In [8]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(2000)),
    eval_dataset=tokenized_datasets["test"].shuffle(seed=42).select(range(1000)),
    tokenizer=tokenizer ,
    compute_metrics=compute_metrics,
)

display(trainer)

# we are choosing a small subset of 2000 samples because the enitre set will take much longer

/tmp/ipython-input-4113579617.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ahsan-shahzad (ahsan-shahzad-foli) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.358938,0.871000
2,0.293700,0.382432,0.904000


TrainOutput(global_step=500, training_loss=0.2936997985839844, metrics={'train_runtime': 451.9713, 'train_samples_per_second': 8.85, 'train_steps_per_second': 1.106, 'total_flos': 1052444221440000.0, 'train_loss': 0.2936997985839844, 'epoch': 2.0})

In [11]:
trainer.evaluate()


{'eval_loss': 0.38243192434310913,
 'eval_accuracy': 0.904,
 'eval_runtime': 29.0445,
 'eval_samples_per_second': 34.43,
 'eval_steps_per_second': 4.304,
 'epoch': 2.0}

In [14]:
import torch


# Put model on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [15]:


def predict_sentiment(test_text):

  # Tokenize and move inputs to GPU
  inputs = tokenizer(test_text, return_tensors="pt", truncation=True, padding=True).to(device)

  # Run model
  outputs = model(**inputs)
  pred = outputs.logits.argmax(-1).item()

  print("Sentiment:", "positive" if pred == 1 else "negative")


In [16]:
text = "I absolutely loved this movie! The acting was fantastic."
predict_sentiment(text)

text2 = "this movie is a waste of time"

predict_sentiment(text2)

Sentiment: positive
Sentiment: negative
